In [6]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(
    temperature=0.1,
)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query : str = Field(
        description="The Query you will search for. Example quesry: Stock Market Symbol for Apple Company"
    )

class StockMarketSymbolSearchTool(BaseTool):
    name : str ="StockMarketSymbolSearchTool"
    description : str = """
    Use this toll to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema : Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol : str = Field(description="Stock symbol of the company. Examples: AAPL, TSLA",)

class CompanyOverviewTool(BaseTool):
    name: str="CompanyOverview"
    description : str = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema : Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=Overview&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyIncomeStatementTool(BaseTool):
    name : str = "CompanyIncomeStatement"
    description : str = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get (f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyStockPerformanceTool(BaseTool):
    name : str = "CompanyStockPerformance"
    description : str = """
    Use this to get the weekly Company stock persformance.
    You should enter a stock symbol
    """
    args_schema : Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=Time_SERIES_MONTHLY_Adjusted&symbol={symbol}&apikey={alpha_vantage_api_key}")

        return r.json()



agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ]
)

prompt = "Give me information on Rigetti Computing Inc's stock, and help me analyze if it's a potential good investment considering its financials, income statement, stock performance. Also tell me what symbol does the stock have."
agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Rigetti Computing Inc'}`


About RGTI. Rigetti Computing, Inc., through its subsidiaries, builds quantum computers and the superconducting quantum processors. The company offers cloud in a form of quantum processing unit, such as 9-qubit chip and Ankaa-2 system under the Novera brand name; and sells access to its quantum computers through quantum computing as a service. Over the last year, Rigetti Computing, Inc.'s stock price has increased by 522.49%. Rigetti Computing, Inc. is currently approximately $11.03 per share. What is Rigetti Computing, Inc.'s 2025 ... Rigetti Computing, Inc. (RGTI.NASDAQ): Stock quote, stock chart, quotes, analysis, advice, financials and news for Stock Rigetti Computing, Inc. | Nasdaq: RGTI | Nasdaq ... Stock Market. Equities. RGTI Stock Best financial portal +951% of historical performance More than 20 years at your side ... Discover re

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 23293 tokens (23019 in the messages, 274 in the functions). Please reduce the length of the messages or functions.